# SeCaPr - Sequence Capture Processor
A computational pipeline for processing Illumina sequence capture data

## Installation & Setup
SeCaPr is available as a conda package on the bioconda channel. This makes installation very simple:

#### 1. Install conda

If you do not have conda installed, got to [this page](https://conda.io/docs/install/quick.html) and follow the instructions for you system. Conda is a software and environment manager, that makes installation of new software and of required dependencies very simple and straightforward.

#### 2. Install the SeCaPr environment (all correct software dependencies)
Conda makes it super easy to install secapr with all necessary dependencies. These will be saved as a conda environment, which you can activate or deactivate. Install the environment with this command and name it however you like, here we name it `secapr_env`:

    conda create -c bioconda -n secapr_env secapr
   
#### 3. Activate the environment
To activate the newly created environment, type:

    source activate secapr_env

When the environment is activated, all the necessary software dependencies will be available in the standarad path, e.g. when you type `samtools` the samtools version required for SeCaPr will be executed. After you are done using secapr, you can deactivate the environment to switch back to your standard environment with this command:

    source deactivate
    
But let's NOT deactivate it for now and continue with the tutorial.

#### 4. Check active environment
Check if you are connected to the correct environment (there should eb a star in front of secapr_env in the output of this command):

    conda info --envs

<div class="alert alert-block alert-info">IMPORTANT : whenever you are using this pipeline, make sure the secapr_env is activated
**(--> simply type `source activate secapr_env` in your command line)**</div>

## Documentation
The documentation is divided into the following steps (click on the links):
- [Cleaning and adapter trimming](docs/notebook/cleaning_trimming.ipynb)
- [De-novo contig assembly](docs/notebook/contig_assembly.ipynb)
- [Extract target contigs](docs/notebook/extract_contigs.ipynb)
- [Align contigs](docs/notebook/align_contigs.ipynb)
- [Reference-based assembly](docs/notebook/reference_assembly.ipynb)
- [Locus selection](docs/notebook/locus_selection.ipynb)
- [Phasing alleles](docs/notebook/phasing.ipynb)